In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import requests
import pandas as pd
import json
from citipy import citipy
import random
from sodapy import Socrata

# Import API key
from config import api_key

In [22]:
df = pd.read_csv("PCS_Locations.csv")
df = df.loc[(df["sensor_id"] == 27) | (df["sensor_id"] == 75) | (df["sensor_id"] == 1),:]
df

,sensor_id,sensor_description,sensor_name,installation_date,status,note,direction_1,direction_2,latitude,longitude,location
46,1,Bourke Street Mall (North),Bou292_T,2009/03/24,A,NaN,West,East,-37.813494,144.965153,"(-37.8134944, 144.96515324)"
65,27,QV Market-Peel St,Vic_T,2013/08/16,A,NaN,East,West,-37.806069,144.956447,"(-37.80606888, 144.95644693)"
73,75,Spring St- Flinders st (West),SprFli_T,2020/12/18,A,NaN,North,South,-37.815153,144.974677,"(-37.81515275, 144.97467662)"


In [109]:
df[["sensor_description","latitude","longitude"]]

,sensor_description,latitude,longitude
46,Bourke Street Mall (North),-37.813494,144.965153
65,QV Market-Peel St,-37.806069,144.956447
73,Spring St- Flinders st (West),-37.815153,144.974677


In [80]:
# Base URL required for API call
url = "https://data.melbourne.vic.gov.au/resource/xt2y-tnn9.geojson?"


In [81]:
# Latitude and Longitudes of Sensor Locations:
lat = list(df["latitude"])
lon = list(df["longitude"])

In [88]:
# Empty list to hold all API calls
response_data = []

In [89]:
# Loop through Open Data Melbourne API, with a radius of 100m
for b in range(len(lat)):
    query_url = f"{url}$where=within_circle(geocoded_column, {lat[b]}, {lon[b]}, 100)"
    response_data.append(requests.get(query_url).json())
    print(lat[b], lon[b])
response_data

-37.813494399999996 144.96515324
-37.80606888 144.95644693
-37.81515275 144.97467662


[{'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [144.965, -37.8131]},
    'properties': {'x_coordinate': '144.965',
     'seating_type': 'Seats - Indoor',
     'industry_anzsic4_description': 'Cafes and Restaurants',
     'property_id': '109284',
     'industry_anzsic4_code': '4511',
     'number_of_seats': '42',
     'census_year': '2017',
     'y_coordinate': '-37.8131',
     'block_id': '55',
     'street_address': '191-205 Swanston Street MELBOURNE 3000',
     'clue_small_area': 'Melbourne (CBD)',
     'base_property_id': '109284',
     'trading_name': 'Soul Cafe'}},
   {'type': 'Feature',
    'geometry': {'type': 'Point', 'coordinates': [144.965, -37.8131]},
    'properties': {'x_coordinate': '144.965',
     'seating_type': 'Seats - Indoor',
     'industry_anzsic4_description': 'Cafes and Restaurants',
     'property_id': '109284',
     'industry_anzsic4_code': '4511',
     'number_of_seats': '42',
     'census_yea

In [155]:
sensor_1 = len(response_data[0]["features"])
sensor_27 = len(response_data[1]["features"])
sensor_75 = len(response_data[2]["features"])
print(sensor_1,sensor_27,sensor_75)
sensor_list = [280, 291, 73]

280 291 73


In [160]:
restr_names = []
seating = []
restr_type = []
lat = []
lon = []

for x in range(len(sensor_list)):
    for a in range(sensor_list[x]):
        restr_names.append(response_data[x]["features"][a]["properties"]["trading_name"])
        seating.append(response_data[x]["features"][a]["properties"]["seating_type"])
        restr_type.append(response_data[x]["features"][a]["properties"]["industry_anzsic4_description"])
        lat.append(response_data[x]["features"][a]["properties"]["y_coordinate"])
        lon.append(response_data[x]["features"][a]["properties"]["x_coordinate"])

In [161]:
new_df = pd.DataFrame({
    "Name": restr_names,
    "Seating Type": seating,
    "Type": restr_type,
    "Latitude": lat,
    "Longitude": lon
})
new_df

,Name,Seating Type,Type,Latitude,Longitude
0,Soul Cafe,Seats - Indoor,Cafes and Restaurants,-37.8131,144.965
1,Soul Cafe,Seats - Indoor,Cafes and Restaurants,-37.8131,144.965
2,Hotel Swanston,Seats - Indoor,Accommodation,-37.8131,144.965
3,Yoyogi Japanese Cafe,Seats - Outdoor,Cafes and Restaurants,-37.8128,144.965
4,Yoyogi Japanese Cafe,Seats - Indoor,Cafes and Restaurants,-37.8128,144.965
...,...,...,...,...,...
639,Nama Nama,Seats - Outdoor,Cafes and Restaurants,-37.8145,144.9741
640,Mercure Hotel,Seats - Indoor,Accommodation,-37.8148,144.9742
641,Kappo,Seats - Indoor,Cafes and Restaurants,-37.8145,144.9741
642,Kappo,Seats - Indoor,Cafes and Restaurants,-37.8145,144.9741


In [163]:
clean_df = new_df.drop_duplicates(subset="Name")
clean_df

,Name,Seating Type,Type,Latitude,Longitude
0,Soul Cafe,Seats - Indoor,Cafes and Restaurants,-37.8131,144.965
2,Hotel Swanston,Seats - Indoor,Accommodation,-37.8131,144.965
3,Yoyogi Japanese Cafe,Seats - Outdoor,Cafes and Restaurants,-37.8128,144.965
5,Crown Cafe Bakery,Seats - Indoor,Cafes and Restaurants,-37.8128,144.965
7,Starbucks Coffee Company,Seats - Indoor,Cafes and Restaurants,-37.8129,144.9658
...,...,...,...,...,...
587,Felt Restaurant,Seats - Indoor,Cafes and Restaurants,-37.8153,144.9736
609,Verge,Seats - Indoor,Cafes and Restaurants,-37.8145,144.9741
611,Verge Restuarant,Seats - Indoor,Cafes and Restaurants,-37.8145,144.9741
635,Nama Nama,Seats - Indoor,Cafes and Restaurants,-37.8145,144.9741


In [164]:
clean_df["Type"].value_counts()

Cafes and Restaurants                                57
Takeaway Food Services                               10
Accommodation                                         7
Internet Service Providers and Web Search Portals     5
Bakery Product Manufacturing (Non-factory based)      2
Technical and Vocational Education and Training       1
Vacant Space                                          1
Pubs, Taverns and Bars                                1
Oil and Gas Extraction                                1
Banking                                               1
Name: Type, dtype: int64

In [165]:
# Save Zomato DataFrame (Consolidated Types)
clean_csv = clean_df.to_csv("Melb_City_Restaurants.csv")